In [4]:
import json

with open('character.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data['Levi Ackerman']['content'].keys()

dict_keys(['Appearance', 'Personality', 'History', "Ilse's Notebook", 'Wall Sealing arc', 'The Female Titan arc', 'Clash of the Titans arc', 'Royal Government arc', 'Return to Shiganshina arc', 'Marley arc', 'War for Paradis arc', 'Bad Boy', 'Vertical Maneuvering Equipment', 'Strength', 'Intelligence', 'Awakened power', 'Sword-Wielding Technique', 'Thunder Spears'])

In [15]:
from bs4 import BeautifulSoup, Tag, NavigableString

html = '''<div class="pi-data-value pi-font">hello<br></div>'''

soup = BeautifulSoup(html, 'html.parser')

In [17]:
value_el = soup.select_one(".pi-data-value")
for ref in value_el.select(".reference"):
    ref.decompose()  # 删除脚注引用
text_lines = []
for elem in value_el.children:
    print(elem, type(elem), elem.name)
    if elem.name == "ul":
        ul_text = "\n".join(li.get_text(" ", strip=True) for li in elem.find_all("li", recursive=False))
        text_lines.append(ul_text)
    elif elem.name:
        text_lines.append(elem.get_text(" ", strip=True))
    elif isinstance(elem, str):
        text_lines.append(elem.strip())
val = "\n".join(filter(None, text_lines))

hello <class 'bs4.element.NavigableString'> None
<br/> <class 'bs4.element.Tag'> br


### analyse valid gt

In [108]:
import json
from collections import defaultdict

def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

path = "./gt/character_fandom.json"
with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)

def split_name_francise(s: str):
    name = s.split('(')[0].strip()
    francise = s.split('(', maxsplit=1)[-1][:-1].strip()
    return name, francise

def save_json(data, path):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

null_dict, invalid_dict = dict(), dict()
fran_counter = defaultdict(int)
incomplete_dict = dict()
for k, item in data.items():
    name, francise = split_name_francise(k)
    if 'name' not in item:
        null_dict[k] = item
        fran_counter[francise] += 1
        continue
    if not item['infobox']:
        incomplete_dict[k] = item

    if name not in item['name']:
        invalid_dict[k] = item
        # fran_counter[francise] += 1


# save_json(null_dict, 'character_null.json')
save_json(invalid_dict, 'character_invalid.json')

print(len(data), len(null_dict), len(invalid_dict))

1082 0 220


In [106]:
save_json(incomplete_dict, "incomplete.json")

In [56]:
## old
print(len(data), len(null_dict), len(invalid_dict))
print(len(fran_counter))

1107 119 295
74


In [109]:
from rapidfuzz import fuzz, process

white = load_json('re_search.json')
will_be_removed = dict()
for char, info in invalid_dict.items():
    if char in white:
        continue
    name, fran = split_name_francise(char)
    name_url = info['url'].split('wiki/')[-1]
    score1 = fuzz.token_sort_ratio(name, info['name'])
    score2 = fuzz.token_sort_ratio(name, name_url.replace('_', ' '))
    if score1 < 80 and score2 < 80:
        will_be_removed[char] = info
        # will_be_removed[char] = {'name':info['name'], 'url': info['url']}
        # will_be_removed[char] = info
        # print((score1, score2, char, info['name'], info['url']))
# save_json(will_be_removed, 'to_be___removed.json')

In [114]:
len(will_be_removed)
save_json(will_be_removed, 'temp.json')

In [14]:
1107-119-335

653

In [82]:
fran_counter2 = defaultdict(int) 
for char in data.keys():
    name, fran = split_name_francise(char)
    fran_counter2[fran] += 1
len(fran_counter2)

397

In [ ]:
fran_counter2['Dragon Ball']

### test google serpapi

In [20]:
import requests
import time

API_KEY = "e83711a1e8dfbb65831919301a04eb7ec78aef55d66d64038e5261bff3122442"  # 替换为你的 API KEY
# CX = "YOUR_SEARCH_ENGINE_ID"     # 替换为你的 CSE ID

def google_serp_search(query):
    """调用 Google Custom Search API 搜索结果"""
    url = "https://serpapi.com/search.json"
    params = {
        "api_key": API_KEY,
        "engine": "google",
        "q": query
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

# query = "Re:ZERO -Starting Life in Another World- site:fandom.com"
# resp = google_serp_search(query)

In [48]:
fran_community_dict = dict()

for fran in fran_counter.keys():
    if fran in fran_community_dict:
        continue
    resp = google_serp_search(f"{fran} site:fandom.com")
    try:
        fran_community_dict[fran] = resp['organic_results']
    except:
        fran_community_dict[fran] = resp['error']
    time.sleep(0.5)

In [47]:
resp

{'search_metadata': {'id': '68f4866648c7910b423fdb64',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/e1756ce4e4428f3c/68f4866648c7910b423fdb64.json',
  'pixel_position_endpoint': 'https://serpapi.com/searches/e1756ce4e4428f3c/68f4866648c7910b423fdb64.json_with_pixel_position',
  'created_at': '2025-10-19 06:34:14 UTC',
  'processed_at': '2025-10-19 06:34:14 UTC',
  'google_url': 'https://www.google.com/search?q=KONOSUBA+-God%27s+blessing+on+this+wonderful+world%21%3A+God%27s+Blessings+On+This+Wonderful+Choker%21+site%3Afandom.com&oq=KONOSUBA+-God%27s+blessing+on+this+wonderful+world%21%3A+God%27s+Blessings+On+This+Wonderful+Choker%21+site%3Afandom.com&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/e1756ce4e4428f3c/68f4866648c7910b423fdb64.html',
  'total_time_taken': 1.26},
 'search_parameters': {'engine': 'google',
  'q': "KONOSUBA -God's blessing on this wonderful world!: God's Blessings On This Wonderful Choker! site:fandom.com"

In [26]:
save_json(fran_community_dict, "fran_community_dict_full.json")

link_dict = dict()
saved_list = ['title', 'link']
saved_num = 5
for k, v in fran_community_dict.items():
    link_dict[k] = []
    for i in range(min(saved_num, len(v))):
        item = {x: v[i][x] for x in saved_list}
        link_dict[k].append(item)

save_json(link_dict, "fran_community_dict_part.json")

In [51]:
import re
from collections import defaultdict
pattern = re.compile(r"https?://([a-zA-Z0-9-]+)\.fandom\.com")

best_link_dict = dict()
for k, v in fran_community_dict.items():
    num_counter = defaultdict(int)
    if not isinstance(v, list):
        print(k, ' not found!!')
        continue
    for item in v:
        match = pattern.search(item['link'])
        com = match.group(1)
        num_counter[com] += 1
    best_com = max(num_counter, key=num_counter.get)
    best_link_dict[k] = f"https://{best_com}.fandom.com"

KONOSUBA -God's blessing on this wonderful world!: God's Blessings On This Wonderful Choker!  not found!!
Persona 5 The Animation -the Day Breakers-  not found!!


In [52]:
save_json(best_link_dict, "fran_community_dict_part.json")

### delete and search individually

In [98]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

data_latest = load_json("./gt/character_fandom_latest.json")
print(len(data_latest))

1107


In [99]:
to_be_removed = load_json('to_be_removed.json')

for k in to_be_removed.keys():
    del data_latest[k]

print(len(data_latest))

1082


In [100]:
from fandom_character_info import parse_character_page

to_be_updated = load_json('re_search.json')
for char, url in to_be_updated.items():
    data_latest[char] = parse_character_page(url)
    print(char, len(data_latest))

Luffy Monkey (ONE PIECE) 1082
Killua Zoldyck (Hunter x Hunter) 1082
Kiyotaka Ayanokouji (Classroom of the Elite) 1082
Jin-U Seong (Solo Leveling) 1082
Yuu Ishigami (Kaguya-sama: Love is War) 1082
Gon Freecss (Hunter x Hunter) 1082
Shinobu Oshino (Nekomonogatari Black) 1082
Spike Spiegel (Cowboy Bebop) 1082
Yuu Nishinoya (HAIKYU!!) 1082
Ace Portgas (ONE PIECE) 1082
Anna Liebert (Monster) 1082
Riki Nendou (The Disastrous Life of Saiki K.) 1082
Gohan Son (Dragon Ball GT) 1082
Chuuya Nakahara (Bungo Stray Dogs) 1082
Shinoa Hiiragi (Seraph of the End: Vampire Reign) 1082
Ken Takakura (Dandadan) 1082
Yuuri Plisetsky (Yuri!!! on ICE) 1082
Chiaki Nanami (Danganronpa 2: Chiaki Nanami's Goodbye Despair Quest) 1082
YoRHa 2-gou B-gata (NieR:Automata: Long Story Short) 1082
Ekubo (Mob Psycho 100) 1082
Jirou Souzousa Shunsui Kyouraku (Bleach) 1082
Tsunade Senju (Naruto) 1082
San (Princess Mononoke) 1082
Fyodor Dostoyevsky (Bungo Stray Dogs 3) 1082
Kyouka Izumi (Bungo Stray Dogs) 1082
Kuro Nero (Blac

In [101]:
save_json(data_latest, "./gt/character_fandom.json")